In [ ]:
# all needed packages
import pandas as pd
from copy import deepcopy
import math as math
import numpy as np
from math import exp
# import argparse
import json,csv, pickle
import itertools,collections
import matplotlib.pyplot as plt

# ML packages
from sklearn.metrics import *
from sklearn.metrics import mean_squared_error, r2_score
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
# from joblib import Parallel, delayed
# import multiprocessing
import xgboost as xgb

# normalization packages
from sklearn.preprocessing import MinMaxScaler, minmax_scale
import scipy
from scipy import *

# DL Keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import skipgrams
from keras.layers import Embedding, Input, Reshape, Dense, concatenate
from keras.models import Sequential, Model

# for protein embedding
import biovec

# for drugs embedding
# import tensorflow as tf
# from tensorflow import keras
# import deepchem as dc
# from deepchem.feat import Featurizer
# from deepchem.models.optimizers import ExponentialDecay, Adam
# from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder
# from deepchem.metrics import to_one_hot
# from deepchem.models.graph_models import GraphConvModel,L2Loss,Dense,Reshape,Dropout 

# Import my files
from training_functions import *
from pathScores_functions import *
from evaluation import *

In [ ]:
# Read Y from the input
fpath = 'Input/Davis/'
affinity = pickle.load(open(fpath + "Y","rb"), encoding='latin1')
affinity = np.array(affinity, dtype=np.float64)

aff = -(np.log10(affinity/(math.pow(10,9))))
aff_mat = (aff - aff.min()) / (aff.max() - aff.min())
#DP = (YY - YY.min()) / (YY.max() - YY.min())

aff_exp = np.exp((-1)*aff)

In [ ]:
from sklearn.preprocessing import normalize

path = '/home/amit/BindingAffinity/EMBED/Davis/'

drEMBED_ = np.load(path + 'unique_drug_Mol2Vec_EMB_DAVIS.npy')
drEMBED1_ = np.load(path + 'unique_drug_GIN_EMB_DAVIS.npy')
drEMBED2_ = np.load(path + 'unique_drug_E3nn_EMB_DAVIS.npy')

drEMBED = normalize(drEMBED_, norm='l2', axis=1)
drEMBED1 = normalize(drEMBED1_, norm='l2', axis=1)
drEMBED2 = normalize(drEMBED2_, norm='l2', axis=1)

prEMBED_ = np.load(path + 'unique_protein_ProVec_EMB_DAVIS.npy')
prEMBED1_ = np.load(path + 'unique_protein_BERT_EMB_DAVIS.npy')
prEMBED2_ = np.load(path + 'unique_protein_ESM_EMB_DAVIS.npy')

prEMBED = normalize(prEMBED_, norm='l2', axis=1)
prEMBED1 = normalize(prEMBED1_, norm='l2', axis=1)
prEMBED2 = normalize(prEMBED2_, norm='l2', axis=1)

drEMBED.shape, prEMBED.shape, drEMBED1.shape, prEMBED1.shape, drEMBED2.shape, prEMBED2.shape

In [ ]:
FV_targets_sq = np.array(prEMBED, dtype = float)
FV_targets_sq1 = np.array(prEMBED1, dtype = float)
FV_targets_sq2 = np.array(prEMBED2, dtype = float)

FV_drugs_sq = np.array(drEMBED, dtype = float)
FV_drugs_sq1 = np.array(drEMBED1, dtype = float)
FV_drugs_sq2 = np.array(drEMBED2, dtype = float)

dr1 = []
dr2 = []
dr3 = []
pr1 = []
pr2 = []
pr3 = []
lab = []

for i,j in itertools.product((range(aff.shape[0])),range(aff.shape[1])):
    #print(i,j)
    dr_1 = list(FV_drugs_sq[i])
    dr_2 = list(FV_drugs_sq1[i])
    dr_3 = list(FV_drugs_sq2[i])
    
    pr_1 = list(FV_targets_sq[j])
    pr_2 = list(FV_targets_sq1[j])
    pr_3 = list(FV_targets_sq2[j])
    
    dr1.append(dr_1)
    dr2.append(dr_2)
    dr3.append(dr_3)
    
    pr1.append(pr_1)
    pr2.append(pr_2)
    pr3.append(pr_3)
    
    label = aff[i][j]
    lab.append(label)
    
dr_emb1 = np.array(dr1)
dr_emb2 = np.array(dr2)
dr_emb3 = np.array(dr3)

pr_emb1 = np.array(pr1)
pr_emb2 = np.array(pr2)
pr_emb3 = np.array(pr3)
    
#XX = np.array(concatenateFV)
label = np.array(lab,dtype=np.float32)
YY = label
#YY = MinMaxScaler(feature_range=(np.min(label), np.max(label))).fit_transform(label.reshape(-1, 1)).flatten()
print('Drug Emb1',dr_emb1.shape)
print('Drug Emb2',dr_emb2.shape)
print('Drug Emb3',dr_emb3.shape)
print('Protein Emb1',pr_emb1.shape)
print('Protein Emb2',pr_emb2.shape)
print('Protein Emb3',pr_emb3.shape)
print('Label shape',YY.shape)

In [ ]:
from keras.models import load_model
from sklearn.model_selection import train_test_split
########################################## Start the Fusion model process ############################################

# Testing evaluation metrics
MSE_test_fusion = []
rm2_test_fusion = []
CI_test_fusion = []
aupr_test_fusion = []
thresh = 7
X = [pr_emb1, pr_emb2, pr_emb3, dr_emb1, dr_emb2, dr_emb3]
Y = YY
kf = KFold(n_splits=5, shuffle=True, random_state=42)
foldCounter = 1  # fold counter

for train_index, test_index in kf.split(X[0]):
    print('===================================================')
    print("** Working with Fold %i : **" % foldCounter)

    x_train = [x[train_index] for x in X]
    x_test = [x[test_index] for x in X]
    y_train, y_test = Y[train_index], Y[test_index]

    # Define the input layers for the three protein embeddings
    protein_input1 = Input(shape=(pr_emb1.shape[1],))
    protein_input2 = Input(shape=(pr_emb2.shape[1],))
    protein_input3 = Input(shape=(pr_emb3.shape[1],))
   
    #protein_concatenated = normalize(protein_concatenated, norm='l2', axis=1)

    protein_bc_fc1 = Dense(1024, activation='relu')(protein_input1)
    protein_bc_fc2 = Dense(1024, activation='relu')(protein_input2)
    protein_bc_fc3 = Dense(1024, activation='relu')(protein_input3)

    protein_concatenated = concatenate([protein_bc_fc1, protein_bc_fc2, protein_bc_fc3])
    #protein_concatenated = concatenate([protein_bc_fc2, protein_bc_fc3])

    

    
    # Define the input layers for the three drug embeddings
    drug_input1 = Input(shape=(dr_emb1.shape[1],))
    drug_input2 = Input(shape=(dr_emb2.shape[1],))
    drug_input3 = Input(shape=(dr_emb3.shape[1],))


    drug_bc_fc1 = Dense(1024, activation='relu')(drug_input1)
    drug_bc_fc2 = Dense(1024, activation='relu')(drug_input2)
    drug_bc_fc3 = Dense(1024, activation='relu')(drug_input3)

    drug_concatenated = concatenate([drug_bc_fc1, drug_bc_fc2, drug_bc_fc3])


    #drug_concatenated = concatenate([drug_bc_fc2, drug_bc_fc3])
    

#     # Define the fully connected layers for each protein input
#    protein_fc1 = Dense(1024, activation='relu')(protein_concatenated)
#     protein_fc2 = Dense(512, activation='relu')(protein_fc1)
#     protein_fc3 = Dense(512, activation='relu')(protein_fc2)

#     # Define the fully connected layers for each drug input
#    drug_fc1 = Dense(1024, activation='relu')(drug_concatenated)
#     drug_fc2 = Dense(512, activation='relu')(drug_fc1)
#     drug_fc3 = Dense(512, activation='relu')(drug_fc2)

    # Concatenate the protein and drug embeddings
    concatenated = concatenate([protein_concatenated, drug_concatenated])

#     n_layers = 4
#     gate = Highway(n_layers = n_layers, value=concatenated, gate_bias=-2)
#     FC1 = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(gate)
#     FC2 = Dropout(0.4)(FC1)
#     FC2 = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(FC2)
#     FC2 = Dropout(0.4)(FC2)
#     FC2 = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(FC2)
# #     FC2 = Dropout(0.3)(FC2) 
    
#     # And add a logistic regression on top
#     output = Dense(1, kernel_initializer='normal')(FC2)




    # Define the output layer
    fc1 = Dense(1024, activation='relu')(concatenated)
    fc1 = Dropout(0.4)(fc1)
    fc2 = Dense(1024, activation='relu')(fc1)
    fc2 = Dropout(0.4)(fc2)
    fc3 = Dense(512, activation='relu')(fc2)
    fc3 = Dropout(0.4)(fc3)
#     fc4 = Dense(1024, activation='relu')(fc3)
#     fc5 = Dense(512, activation='relu')(fc4)
    output = Dense(1, activation='linear')(fc3)

    

    # Define the model
    model = Model(inputs=[protein_input1, protein_input2, protein_input3, drug_input1, drug_input2, drug_input3], outputs=output)

    #model = Model(inputs=[protein_input2, protein_input3, drug_input2, drug_input3], outputs=output)

    #metrics = ['mse']
    # Compile the model
    learning_rate = 0.0001
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    # Train the model with early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0)
    model.fit(x=x_train, y=y_train, epochs=500, batch_size=64, verbose=0, validation_split=0.1, callbacks=[early_stopping])# Save the model for the current fold
    model.save(f'model_fold_{foldCounter}.h5')

    # Predict the outputs for the test data
    predictedY_test = model.predict(x_test)
    predictedY_test = predictedY_test.flatten()

    print("--------------- Test Set Evaluation #----------------")

    print('Mean Squared Error: %.4f' % mean_squared_error(y_test, predictedY_test))
    MSE_test_fusion.append(mean_squared_error(y_test, predictedY_test))

    print('Concordance index: %4f' % concordance_index(y_test, predictedY_test))
    CI_test_fusion.append(concordance_index(y_test, predictedY_test))

    print('aupr: %.4f' % get_aupr(y_test, predictedY_test, thresh))
    aupr_test_fusion.append(get_aupr(y_test, predictedY_test, thresh))

    print('rm2: %.4f' % get_rm2(y_test, predictedY_test))
    rm2_test_fusion.append(get_rm2(y_test, predictedY_test))

    foldCounter += 1


In [ ]:
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, concordance_index

# Lists to store ground truth and predicted values
ground_truth_values = []
predicted_values = []

# Testing evaluation metrics
MSE_test_fusion = []
CI_test_fusion = []
aupr_test_fusion = []
rm2_test_fusion = []
thresh = 7

# Assuming pr_emb1, pr_emb2, ..., YY are your input features and target variable
X = [pr_emb1, pr_emb2, pr_emb3, dr_emb1, dr_emb2, dr_emb3]
Y = YY

kf = KFold(n_splits=5, shuffle=True, random_state=42)

foldCounter = 1  # fold counter

for train_index, test_index in kf.split(X[0]):
    print('===================================================')
    print("** Working with Fold %i : **" % foldCounter)

    x_train = [x[train_index] for x in X]
    x_test = [x[test_index] for x in X]
    y_train, y_test = Y[train_index], Y[test_index]

    # Load the saved model
    saved_model_path = f'model_fold_{foldCounter}_DAVIS.h5'
    model = load_model(saved_model_path)

    # Predict the outputs for the test data
    predictedY_test = model.predict(x_test)
    predictedY_test = predictedY_test.flatten()

    # Store ground truth and predicted values
    ground_truth_values.extend(y_test)
    predicted_values.extend(predictedY_test)

    print("--------------- Test Set Evaluation # ----------------")

    print('Mean Squared Error: %.4f' % mean_squared_error(y_test, predictedY_test))
    MSE_test_fusion.append(mean_squared_error(y_test, predictedY_test))

    print('Concordance index: %4f' % concordance_index(y_test, predictedY_test))
    CI_test_fusion.append(concordance_index(y_test, predictedY_test))

    # Assuming you have functions get_aupr and get_rm2 defined somewhere
    print('aupr: %.4f' % get_aupr(y_test, predictedY_test, thresh))
    aupr_test_fusion.append(get_aupr(y_test, predictedY_test, thresh))

    print('rm2: %.4f' % get_rm2(y_test, predictedY_test))
    rm2_test_fusion.append(get_rm2(y_test, predictedY_test))

    foldCounter += 1

# Create a scatter plot of predicted vs. ground truth values
plt.scatter(ground_truth_values, predicted_values, label='Predicted vs. Ground Truth', s=10)
plt.xlabel('Ground Truth Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs. Ground Truth Values')
plt.legend()
plt.show()
plt.savefig('scatter_plot.png', format='png')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

slope, intercept = np.polyfit(predicted_values, ground_truth_values, 1)
plt.figure(figsize=(8, 6))
plt.scatter(predicted_values, ground_truth_values, s=10)
plt.plot(predicted_values, slope * np.array(predicted_values) + intercept, color='red', linestyle='--')

plt.ylabel('Ground Truth Values')
plt.xlabel('Predicted Values')
plt.show()
